In [1]:
import os
import sqlite3
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [2]:
# Embeddings configurados
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/home/alee/.local/share/virtualenvs/chronosAI-bGPFpNdy/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ========== 1. CARGAR DOCUMENTO MARKDOWN ==========

In [3]:
try:
    file_path = 'data/politicaNominas.md'
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Archivo no encontrado: {file_path}")

    with open(file_path, "r", encoding="utf-8") as archivo:
        contenido = archivo.read()
        print(f"✅ Archivo Markdown leído con éxito: {file_path}")

    headers_to_split_on = [("#", "Header"), ("##", "Title")]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(contenido)

    vector_store_md = Chroma.from_documents(
        md_header_splits,
        embeddings,
        collection_metadata={"hnsw:space": "cosine"},
        persist_directory="data/stores/nominas",
    )
    print("✅ Vector store del archivo Markdown creado con éxito.")

except Exception as e:
    print(f"❌ Error procesando el archivo Markdown: {e}")

✅ Archivo Markdown leído con éxito: data/politicaNominas.md
✅ Vector store del archivo Markdown creado con éxito.


### ========== 2. CARGAR DATOS DE SQLITE ==========

In [4]:
try:
    DATABASE_PATH = 'data/empleados.db'
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()

    # Obtener columnas
    cursor.execute("PRAGMA table_info(empleados)")
    columnas = [col[1] for col in cursor.fetchall()]
    if not columnas:
        raise Exception("No se encontraron columnas en la tabla 'empleados'.")
    print("✅ Columnas encontradas:", columnas)

    # Leer todos los datos
    query = "SELECT * FROM empleados"
    cursor.execute(query)
    data = cursor.fetchall()
    if not data:
        raise Exception("La tabla 'empleados' está vacía.")

    # Crear objetos Document
    documents = []
    for row in data:
        content = ", ".join([f"{columnas[i]}: {row[i]}" for i in range(len(columnas))])
        # Filtrar valores None del metadata
        metadata = {k: v for k, v in zip(columnas, row) if v is not None}
        documents.append(Document(page_content=content, metadata=metadata))

    conn.close()

    # Crear el vector store
    vector_store_sql = Chroma.from_documents(
        documents,
        embeddings,
        collection_metadata={"hnsw:space": "cosine"},
        persist_directory="data/stores/empleados",
    )
    print("✅ Vector store de empleados creado con éxito.")

except sqlite3.Error as e:
    print(f"❌ Error en la base de datos: {e}")
except Exception as e:
    print(f"❌ Error general: {e}")


✅ Columnas encontradas: ['id_empleado', 'nombre', 'fecha_alta', 'departamento', 'cargo', 'salario_base', 'antiguedad', 'complemento_antiguedad', 'complemento_peligrosidad', 'complemento_nocturnidad', 'complemento_turnicidad', 'plus_transporte', 'horas_extras_mes_anterior', 'dietas_mes_anterior', 'porcentaje_irpf', 'pagas_extras', 'prorrateo_pagas_extras', 'cuenta_bancaria', 'convenio_colectivo', 'observaciones']
✅ Vector store de empleados creado con éxito.
